**Update**

We've included `Bayesian Optimization` from [this notebook](https://www.kaggle.com/steubk/simple-oof-ensembling-methods-for-classification). Thanks to [steubk](https://www.kaggle.com/steubk). Aim was to compare with `scipy.optimize` function. 

---

In this notebook, we attempt to find the best weight vector for ensembling via trying to maximize **OOF CV** scores. The piece of code actually came from [this work](https://www.kaggle.com/tilii7/cross-validation-weighted-linear-blending-errors). 


**Disclaimer**

All the input scripts are designed for demonstratio purpose and also the procedure tha we've presented here. Please, use it to your own risk. Also note, in order to use it, you Have-To-Have **N** times `oof.csv` and `test.csv` that trained one the **same validation fold**. Thanks.

---

Set Up:

```
Model: E6
Seed : 42
Exp  : 5 Fold Training, Total 4 experiment [same fold trainig]
```

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from bayes_opt import BayesianOptimization
from scipy.optimize import minimize
from sklearn.metrics import roc_auc_score

dir = "../input/e6-oof-prediction//"
np.random.seed(42)

# exp 1 ___________
oof_one = pd.read_csv(dir + "oof_e6_exp1_seed_42.csv")
test_one = pd.read_csv(dir + "s_e6_exp1_seed_42.csv")
oof_one = oof_one.sort_values(by=["image_name"], ascending=True).reset_index(drop=True)
test_one = test_one.sort_values(by=["image_name"], ascending=True).reset_index(
    drop=True
)

# exp 2 ___________
oof_two = pd.read_csv(dir + "oof_e6_exp2_seed_42.csv")
test_two = pd.read_csv(dir + "s_e6_exp2_seed_42.csv")
oof_two = oof_two.sort_values(by=["image_name"], ascending=True).reset_index(drop=True)
test_two = test_two.sort_values(by=["image_name"], ascending=True).reset_index(
    drop=True
)

# exp 3 ___________
oof_three = pd.read_csv(dir + "oof_e6_exp3_seed_42.csv")
test_three = pd.read_csv(dir + "s_e6_exp3_seed_42.csv")
oof_three = oof_three.sort_values(by=["image_name"], ascending=True).reset_index(
    drop=True
)
test_three = test_three.sort_values(by=["image_name"], ascending=True).reset_index(
    drop=True
)

# exp 4 ___________
oof_four = pd.read_csv(dir + "oof_e6_exp4_seed_42.csv")
test_four = pd.read_csv(dir + "s_e6_exp4_seed_42.csv")
oof_four = oof_four.sort_values(by=["image_name"], ascending=True).reset_index(
    drop=True
)
test_four = test_four.sort_values(by=["image_name"], ascending=True).reset_index(
    drop=True
)


In [ ]:
oof_one.fold.value_counts()

In [ ]:
oof_one.head()

In [ ]:
test_one.head()

# Scipy Optimizer

Let's try first optmization function from `Scipy`, usng `L-BFGS-B` method. We've tried other methods, but this one gave best.

In [ ]:
blend_train = []
blend_test = []

# out of fold prediction
blend_train.append(oof_one.pred)
blend_train.append(oof_two.pred)
blend_train.append(oof_three.pred)
blend_train.append(oof_four.pred)
blend_train = np.array(blend_train)

# submission scores
blend_test.append(test_one.target)
blend_test.append(test_two.target)
blend_test.append(test_three.target)
blend_test.append(test_four.target)
blend_test = np.array(blend_test)

In [ ]:
a = 0.12345
print("{0:.2f}".format(a))

In [ ]:
def roc_min_func(weights):
    final_prediction = 0
    for weight, prediction in zip(weights, blend_train):
        final_prediction += weight * prediction
    return roc_auc_score(np.array(oof_one.target), final_prediction)


print("\n Finding Blending Weights ...")
res_list = []
weights_list = []

for k in range(100):
    starting_values = np.random.uniform(size=len(blend_train))
    bounds = [(0, 1)] * len(blend_train)

    res = minimize(
        roc_min_func,
        starting_values,
        method="L-BFGS-B",
        bounds=bounds,
        options={"disp": False, "maxiter": 100000},
    )

    res_list.append(res["fun"])
    weights_list.append(res["x"])

    
    res_fun = format(res["fun"], '.7f')
    items = [format(item, '.7f') for item in res["x"]]
    
    print(
        "{iter}\tScore: {score}\tWeights: {weights}".format(
            iter=(k + 1),
            score=res_fun,
            weights="\t".join([str(item) for item in items]),
        )
    )

bestSC = np.max(res_list)
bestWght = weights_list[np.argmax(res_list)]
weights = bestWght
blend_score = round(bestSC, 6)

In [ ]:
print('\n Ensemble Score: {best_score}'.format(best_score=bestSC))
print('\n Best Weights: {weights}'.format(weights=bestWght))

train_prices = np.zeros(len(blend_train[0]))
test_prices  = np.zeros(len(blend_test[0]))

print('\n Your final model:')
for k in range(len(blend_test)):
    print(' %.6f * model-%d' % (weights[k], (k + 1)))
    test_prices += blend_test[k] * weights[k]

for k in range(len(blend_train)):
    train_prices += blend_train[k] * weights[k]

In [ ]:
test_one.target = (
    test_one.target.values * bestWght[0]
    + test_two.target.values * bestWght[1]
    + test_three.target.values * bestWght[2]
    + test_four.target.values * bestWght[3]
) / max(bestWght)


In [ ]:
plt.hist(test_one.target,bins=100)
plt.ylim((0,100))
plt.show()

# Bayesian Optimization

Next, Bayesian. 

In [ ]:
train = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/train.csv')
test = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/test.csv')
sub = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/sample_submission.csv')

In [ ]:
dirname = '/kaggle/input/e6-oof-prediction'
prefix = ['exp1', 'exp2', 'exp3', 'exp4', ]

for model in range(4):
    _oof = pd.read_csv(os.path.join(dirname, f"oof_e6_{prefix[model]}_seed_42.csv"))
    score = roc_auc_score(_oof['target'], _oof['pred'])
    print(f"{prefix[model]}: OOF auc:{score:.4}")

    _oof = _oof.rename(columns={"pred":prefix[model]}).drop(["target"],axis=1)
    
    if "fold" in _oof.columns:
        _oof = _oof.drop(["fold"],axis=1)

    train = train.merge(_oof, on="image_name")   

    _sub = pd.read_csv(os.path.join(dirname, f"s_e6_{prefix[model]}_seed_42.csv"))
    _sub.columns = ["image_name", prefix[model]]    
    test = test.merge(_sub, on="image_name")  

In [ ]:
def dim_optimizer (df_oof, features, init_points = 20, n_iter = 100 ):
    pbounds = {'c0': (0.0, 1.0), 
               'c1': (0.0, 1.0), 
               'c2': (0.0, 1.0), 
               'c3': (0.0, 1.0)}
    
    features = features

    def dim_opt (df_oof, c0,c1,c2,c3):

        x = (c0*df_oof[features[0]] + 
             c1*df_oof[features[1]] + 
             c2*df_oof[features[2]] + 
             c3*df_oof[features[3]])
        
        return roc_auc_score(df_oof['target'], x)



    def q (c0,c1,c2,c3):
        return dim_opt  ( df_oof, c0,c1,c2,c3)

    optimizer = BayesianOptimization(
        f=q,
        pbounds=pbounds,
        random_state=42,
    )


    optimizer.maximize(
        init_points=init_points,
        n_iter=n_iter,
    )

    c0 = optimizer.max["params"]["c0"]
    c1 = optimizer.max["params"]["c1"]
    c2 = optimizer.max["params"]["c2"]
    c3 = optimizer.max["params"]["c3"]
    t  = optimizer.max["target"]
    
    print ( f'bo auc:{t}, c0:{c0}, c1:{c1}, c2:{c2}, c3:{c3}' )
    
    return c0, c1, c2, c3


c0, c1, c2, c3 = dim_optimizer (train, prefix, 
                                init_points = 40, 
                                n_iter = 40  )

print(' ')
print (prefix[0],c0)
print (prefix[1],c1)
print (prefix[2],c2)
print (prefix[3],c3)

In [ ]:
def bo_pred(df):
    x = (
        c0 * df[prefix[0]]
        + c1 * df[prefix[1]]
        + c2 * df[prefix[2]]
        + c3 * df[prefix[3]]
    )

    return x


train["pred"] = bo_pred(train)
score = roc_auc_score(train["target"], train["pred"])
print(f"auc bo:{score}")


In [ ]:
plt.hist(train.pred,bins=100)
plt.ylim((0,100))
plt.show()

## Compare L-BFGS-B vs Bayesian Operations


|  Method | CV  |  
|---|---|
| L-BFGS-B  | 0.9316942291023538  |   
| Bayesian-Op  | 0.9316718240316467  |